# 📊 NVIDIA Stock Analysis Around GPU Series Launches
This notebook analyzes the impact of major NVIDIA GPU launches (RTX 20, 30, 40, 50 series) on its stock price.  
We study how the stock performs **5 months before and after** each launch, using event-driven financial analysis.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Load CSV
df = pd.read_csv("Nvidia_stock_data.csv")
df['Date'] = pd.to_datetime(df['Date'])
df.head()

In [ ]:
# GPU Series Launch Dates (Launch Announce)
launch_dates = {
    "RTX 20": datetime(2018, 8, 20),
    "RTX 30": datetime(2020, 9, 1),
    "RTX 40": datetime(2022, 9, 20),
    "RTX 50": datetime(2025, 1, 6)
}

# Filter ±5 months (To analyze before and after trend )
five_months = timedelta(days=150)
filtered = []

for series, launch in launch_dates.items():
    temp_df = df[(df['Date'] >= launch - five_months) & (df['Date'] <= launch + five_months)].copy()
    temp_df['Series'] = series
    temp_df['Phase'] = temp_df['Date'].apply(lambda x: 'Pre-launch' if x < launch else 'Post-launch')
    temp_df['Days_From_Launch'] = (temp_df['Date'] - launch).dt.days
    pre_avg = temp_df[temp_df['Phase'] == 'Pre-launch']['Close'].mean()
    temp_df['Close_Diff'] = temp_df['Close'] - pre_avg
    filtered.append(temp_df)

final_df = pd.concat(filtered).reset_index(drop=True)

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(14, 6))
sns.lineplot(data=final_df, x="Days_From_Launch", y="Close_Diff", hue="Series", style="Phase")
plt.axhline(0, color='black', linestyle='--', linewidth=1)
plt.title("NVIDIA GPU Series: Price Change Relative to Pre-launch Average")
plt.xlabel("Days from Launch (Negative = Pre, Positive = Post)")
plt.ylabel("Close - Pre-launch Avg Close")
plt.tight_layout()
plt.show()

##  Interpretation

- **RTX 50**: Highest volatility after launch, stock itself saw a lot of dips due to maybe overvaluation, Deepseek R1 announcment or even hardware issues of GPUs.

- **RTX 40**: Flat — minimal investor reaction, not much difference in pre-launch or post-launch trends, maybe due to crypto crash or due to market and investor deviation turning to GPUs like A100 which help with cloud and AI.

- **RTX 30**: Positive performance — triggered a very positive reposnse, causes might be linked to pandemic, higher GPU demands by crypto miners, limited initial volume created a artificial scarcity, gaming in general also saw a rise.

- **RTX 20**: Dipped before launch, recovered after — Nvidia saw initial overestimation of Gtx series and RTX wasnt well recieved at first but later on after games introduced ray tracing, it became a long term bet and inventory issues were also resolved.

In [ ]:
# Save for reuse
final_df.to_csv("Nvidia_GPU_Event_Analysis.csv", index=False)